# Could Star Wars planets exist in our universe?

## Part 2. Data cleaning

<img src="https://www.cnet.com/a/img/5WSMXSqEJeSmYVfAfN_SnzfM-l4=/1200x675/2015/01/21/91c68cb0-a2dd-4192-ba46-b847854767d5/luke-works-on-r2.jpg">
Source: www.cnet.com

### Table of contents:
1. Introduction
2. Goal
3. Data cleaning  
    3.1. Exoplanets  
    3.2. Star Wars Planets
4. Summary

## 1. Introduction

Welcome to the part 2 of our journey in search for the answer to the question: "could Star Wars planets exist in our universe"?

Our goal is to find corresponding data concerning our real life exoplanets and fictional Star Wars planets, compare them and create an algorithm that will find the most similar planet to the chosen one.

In the previous part of this project I've managed to scrape data about exoplanets and Star Wars planets from wikipedia and wookieepedia sites, convert them into dataframes and save as csv files.

Now it's time to do some cleaning.

## 2. Goal

The data we've collected is in really "raw" state - it is set of string objects pulled directly from wiki sites and put into adequate places. In order to make operations on this data we need to clean them first. This is our plan:
- define units that describes our values
- check if units duplicate and what is their priority
- create units columns for corresponding values
- clean value coulmn from every character that is not a number (or a decimal point)
- convert value column from string to float
- find common units for both data sets
- convert value columns to base unit based on unit column
- get rid of redundand columns, standardize column names
- save dataframes to csv files

## 2. Data cleaning

In [1]:
import numpy as np
import pandas as pd

### 2.1. Exoplanets

Let's start with importing exoplanet data and some basic cleaning operations - dropping rows containing only NaN values and replacing remaining NaN values as empty strings.

In [2]:
exoPlanets = pd.read_csv('./Exoplanets.csv', index_col=0)
exoPlanets = exoPlanets.dropna(axis=0, how='all').fillna('')
exoPlanets.head()

,Orbital period,Mean radius
Kepler-42c,0.45328731±5*10−8 d,0.73±0.03 REarth
55 Cancri e,0.7365474 (± 0.0000014) d17.677 h,1.875 ± 0.029 REarth
WASP-19b,0.78884 ± 0.0000003 d (18.9321600 ± 7.2×10−6 h...,1.386±0.032 RJ
WASP-43b,0.81347753 (± 0.00000071) d,1.04 +0.07−0.09 RJ
Kepler-10b,0.837495 d20.0999 h,1.47+0.03−0.02 REarth


We will start with orbital period column. Units present there describe time period in which planet orbits around it's star. In the head we already see hours and days. Another units that could describe time periods would be months and years (it's highly unlikely to see minutes and seconds here, but we will check that in later steps). Let's see how many representations of those units we can find in Orbital period column.

In [3]:
h = len(exoPlanets[exoPlanets['Orbital period'].str.contains('h')])
d = len(exoPlanets[exoPlanets['Orbital period'].str.contains('d')])
m = len(exoPlanets[exoPlanets['Orbital period'].str.contains('m')])
y = len(exoPlanets[exoPlanets['Orbital period'].str.contains('y')])
print('''Orbital period units:
{} hours
{} days
{} months
{} years'''.format(h,d,m,y))

Orbital period units:
16 hours
471 days
0 months
161 years


Based on this info we can skip months in our calculations and focus on hours, days and years.

There is a catch though. If you watch closely at our dataframe head you can notice that there are some cells with multiple units. We need to make sure which unit has priority.

In order to do that we will check several things. First of all - how many **days** are in cells with **hours** units?

In [4]:
orbitalH = exoPlanets[exoPlanets['Orbital period'].str.contains('h')]
orbitalDH = orbitalH[orbitalH['Orbital period'].str.contains('d')]
print('''hours: {}
days in hours: {}'''.format(orbitalH.size, orbitalDH.size))

hours: 32
days in hours: 32


Interesting. All cells with hours units have also days units. Which one is more important to us?

It's simple. Since it mathematically doesn't matter (after choosing base unit both values after conversion will give us the same outcome) we will choose the one that is first (closest to the left side). This way it will be much easier to split string leaving desirable value.

In [5]:
before = []
after = []
for i in orbitalDH['Orbital period']:
    if i.find('d') < i.find('h'):
        before.append(i)
    else:
        after.append(i)
print('''days before hours: {}
days after hours: {}'''.format(len(before),len(after)))

days before hours: 16
days after hours: 0


Our prime unit here is **days**. That means that **hours** unit is irrelevant, because in every cell that contains it, the **days** is the main unit anyway.

Now let's do the same with **years** unit.

In [6]:
orbitalY = exoPlanets[exoPlanets['Orbital period'].str.contains('y')]
orbitalDY = orbitalY[orbitalY['Orbital period'].str.contains('d')]
print('''years: {}
days in years: {}'''.format(orbitalY.size, orbitalDY.size))

years: 322
days in years: 292


We can see, that there are some Orbital periods described only by **years** unit. What about double units?

In [7]:
before = []
after = []
for i in orbitalDY['Orbital period']:
    if i.find('d') < i.find('y'):
        before.append(i)
    else:
        after.append(i)
print('''days before years: {}
days after years: {}'''.format(len(before),len(after)))

days before years: 146
days after years: 0


Same as before. **Days** is the main unit if present.

Lastly let's check if we didn't miss anything - are there orbital periods that are not descibed by **hours**, **days** or **years**?

In [8]:
orbitalNone = exoPlanets[~exoPlanets['Orbital period'].str.contains('d|y|h')]
orbitalNone

,Orbital period,Mean radius
Kepler-40b,,1.17 (± 0.04) RJ13.12 REarth
ROXs 42Bb,,0.9 – 3 RJ


Nope, there are none.

Conclusion: **Days** is the primary unit in determining orbital period.

Now we need to create an adequate units column.

In [9]:
exoPlanets.insert(loc=1, column='OrbitalPeriodUnit',
                  value=exoPlanets['Orbital period'].apply(lambda i: i if i=='' else 'days' if 'd' in str(i) else 'years'))
exoPlanets.head()

,Orbital period,OrbitalPeriodUnit,Mean radius
Kepler-42c,0.45328731±5*10−8 d,days,0.73±0.03 REarth
55 Cancri e,0.7365474 (± 0.0000014) d17.677 h,days,1.875 ± 0.029 REarth
WASP-19b,0.78884 ± 0.0000003 d (18.9321600 ± 7.2×10−6 h...,days,1.386±0.032 RJ
WASP-43b,0.81347753 (± 0.00000071) d,days,1.04 +0.07−0.09 RJ
Kepler-10b,0.837495 d20.0999 h,days,1.47+0.03−0.02 REarth


First part is behind us. Now we need to clear value column. To do that we will create a list of all characters that are not a number and exclude decimal point from this list. This will be our **"banned" list**.

In [10]:
notNum=[]
orb = exoPlanets['Orbital period']
for per in orb:
    for i in range(len(per)):
        if str(per)[i] not in '1234567890':
            notNum.append(per[i])
notNum = pd.Series(notNum)
notNumList = list(notNum.unique())
notNumList.remove('.')

And now we will use this set to create a function that will filter redundand elements and extract values. Keep in mind that value the most to the left is our desired one. We will used this knowledge to split strings.

In [11]:
def clear_period(period):
    #splitting string to keep value the most to the left side
    trimmedPeriod = str(period).split('d')[0].split('y')[0].split('±')[0].split('+')[0].split('-')[0].split(' ')[0]
    clearedPeriod = []
    for i in range(len(trimmedPeriod)):
        #extracting only characters that are not in our "banned" list
        if trimmedPeriod[i] not in notNumList:
            clearedPeriod.append(trimmedPeriod[i])
    return ''.join(clearedPeriod)

The only thing left is to apply our function to **Orbital period** column.

In [12]:
exoPlanets['Orbital period'] = exoPlanets['Orbital period'].apply(clear_period)
exoPlanets.head()

,Orbital period,OrbitalPeriodUnit,Mean radius
Kepler-42c,0.45328731,days,0.73±0.03 REarth
55 Cancri e,0.7365474,days,1.875 ± 0.029 REarth
WASP-19b,0.78884,days,1.386±0.032 RJ
WASP-43b,0.81347753,days,1.04 +0.07−0.09 RJ
Kepler-10b,0.837495,days,1.47+0.03−0.02 REarth


At first glance looks fine. Let's try to convert this column to float type.

In [13]:
try:
    exoPlanets['Orbital period'].apply(lambda i: i if i=='' else float(i))
except Exception as e:
    print(e)

could not convert string to float: '9.61849.61359.6234'


Not quite as planned. But worry not, there is no problem that can't be solved.

I hope.

Apparently there are values with multiple dots (decimal points). First of all we need to determine how many such values we have in our data set. To do this we will define a function checking number of dots and create a column that will return number of dots in value column. We will filter by this column to check those duplicates.

In [14]:
def dots_duplicates(period):
    dots = 0
    for i in range(len(str(period))):
        if period[i] =='.':
            dots+=1
    if dots>1:
        return 'Dots duplicated'
    return 'No duplicates'

In [15]:
exoPlanets.insert(loc=2, column='DotsDuplicates', value=exoPlanets['Orbital period'].apply(dots_duplicates))
exoPlanets.query('DotsDuplicates == "Dots duplicated"')

,Orbital period,OrbitalPeriodUnit,DotsDuplicates,Mean radius
HD 40307 c,9.61849.61359.6234,days,Dots duplicated,
HD 40307 d,20.43220.40820.454,days,Dots duplicated,


There are only two such values. After checking wiki sites of <a href="https://en.wikipedia.org/wiki/HD_40307_c">HD 40307 c</a> and <a href="https://en.wikipedia.org/wiki/HD_40307_d">HD 40307 d</a> I noticed, that the author wanted to show value deviation by writing down three values - main one, lower limit (as lower index) and upper limit (as upper index). This is why all of those values were joined together. Unfortunately I don't think there is a simple way to automate clearing process in this particular case, so this time I will do an exception and correct those values manually.

In [16]:
exoPlanets.replace({'Orbital period' : {'9.61849.61359.6234':'9.6184', '20.43220.40820.454':'20.432'}}, inplace=True)
exoPlanets.query('DotsDuplicates == "Dots duplicated"')

,Orbital period,OrbitalPeriodUnit,DotsDuplicates,Mean radius
HD 40307 c,9.6184,days,Dots duplicated,
HD 40307 d,20.432,days,Dots duplicated,


Much better. Now we can get rid of **DotsDuplicates** column and finally convert **Orbital period** column to float type.

In [17]:
exoPlanets.drop(columns='DotsDuplicates', inplace=True)
exoPlanets['Orbital period'] = exoPlanets['Orbital period'].apply(lambda i: i if i=='' else float(i))
exoPlanets.head()

,Orbital period,OrbitalPeriodUnit,Mean radius
Kepler-42c,0.453287,days,0.73±0.03 REarth
55 Cancri e,0.736547,days,1.875 ± 0.029 REarth
WASP-19b,0.78884,days,1.386±0.032 RJ
WASP-43b,0.813478,days,1.04 +0.07−0.09 RJ
Kepler-10b,0.837495,days,1.47+0.03−0.02 REarth


Now we will do the same with **Mean radius** column.

We see two different units in dataframe head - **RE** and **RJ**. Those refer to **Earth Radius** and **Jupiter Radius**. Let's see if there are others.

In [18]:
noRadius = exoPlanets[~exoPlanets['Mean radius'].str.contains('RJ|RE')]
noRadius[noRadius['Mean radius']!='']

,Orbital period,OrbitalPeriodUnit,Mean radius
Kepler-25b,6.238297,days,2.748+0.038−0.035 R🜨
Kepler-25c,12.7207,days,5.217+0.070−0.065 R🜨
Kepler-560b,18.47762,days,1.93+0.15−0.14 R🜨
HD 20781 c,85.131,days,2.17 R+


There are two more units. After checking wiki, I found out that both of those symbols means Earth Radius.

This will make things a little bit easier. Mean radius will be either **RE** or **RJ**.

How many planets are described with both of this units though?

In [19]:
exoPlanets[exoPlanets['Mean radius'].str.contains('RJ')&exoPlanets['Mean radius'].str.contains('RE')]

,Orbital period,OrbitalPeriodUnit,Mean radius
COROT-7b,0.853585,days,0.14 RJ1.58 ± 0.1 REarth
Kepler-4b,3.2135,days,0.357 RJ3.878 REarth
Kepler-5b,3.54846,days,1.431 ± 0.048 RJ15.5457 REarth
Kepler-40b,,,1.17 (± 0.04) RJ13.12 REarth
Kepler-34b,288.822,days,0.764 +0.0012−0.0014 RJ8.56 REarth
PSR J1719-1438 b,0.090706,days,≤0.4 RJ≤4 REarth


Luckily not that many. This time we don't need to create loop to see, that RJ values are before RE values in all rows, where both are present. Therefore RJ will be our priority unit.

Let's repeat same steps as before.

In [20]:
#inserting units column
exoPlanets.insert(loc=3, column='RadiusUnit',
                  value=exoPlanets['Mean radius'].apply(lambda i: i if i=='' else 'RJ' if 'RJ' in str(i) else 'RE'))

In [21]:
#creating banned list
notNum2=[]
meanRad = exoPlanets['Mean radius']
for rad in meanRad:
    for i in range(len(rad)):
        if str(rad)[i] not in '1234567890':
            notNum2.append(rad[i])
notNum2 = pd.Series(notNum2)
notNumList2 = list(notNum2.unique())
notNumList2.remove('.')

In [22]:
#difining clearing function
def clear_radius(radius):
    trimmedRadius = str(radius).split('R')[0].split('±')[0].split('+')[0].split('-')[0].split(' ')[0]
    clearedRadius = []
    for i in range(len(trimmedRadius)):
        if trimmedRadius[i] not in notNumList2:
            clearedRadius.append(trimmedRadius[i])
    return ''.join(clearedRadius)

In [23]:
#applying clearing function
exoPlanets['Mean radius'] = exoPlanets['Mean radius'].apply(clear_radius)
#converting values to float
exoPlanets['Mean radius'] = exoPlanets['Mean radius'].apply(lambda i: i if i=='' else float(i))
exoPlanets.head()

,Orbital period,OrbitalPeriodUnit,Mean radius,RadiusUnit
Kepler-42c,0.453287,days,0.73,RE
55 Cancri e,0.736547,days,1.875,RE
WASP-19b,0.78884,days,1.386,RJ
WASP-43b,0.813478,days,1.04,RJ
Kepler-10b,0.837495,days,1.47,RE


That's it. We have complete and cleaned data for Exoplanets. Time for Star Wars planets.

### 2.2. Star Wars Planets

In [24]:
swPlanets = pd.read_csv('./StarWarsPlanets.csv', index_col=0)
swPlanets = swPlanets[['lengthyear','lengthday','diameter']].dropna(how='all').fillna('')
swPlanets

,lengthyear,lengthday,diameter
Abafar,,,12 11 714 (polar diameter)
Alderaan,364 standard days,18 standard hours,"12,500 kilometers"
Aleen,399 days,31 hours,
Asmeru,At least sixteen months,,
Atollon,505 standard days,29.5 standard hours,
Auratera,280 days per year,18 hours per day,
Bardotta,608 days,25 hours,
Batuu,365 days,24 hours,"12,750 kilometers"
Bespin,14 standard-years,12 standard-hours,"118,000 kilometers, (73,322 miles)"
Cato Neimoidia,278 local days,25 standard hours,


This data set is short enough for us to see all of it's content. This will allow us to detect some elements automatically to some extent - we won't need to create functions and loops to check all of the data stored in the set. In larger datasets this wouldn't be possible, but we will adapt to the existing situation making sure that data is read, interpreted and processed correctly.

In short terms - just don't try this on a larger sets.

You are also probably wondering why do we need lengthday column. I will explain this later.

First of all - I noticed that most diameter values are described by kilometers. Let's check those, which are not.

In [25]:
swPlanets[~swPlanets['diameter'].str.contains('k')]

,lengthyear,lengthday,diameter
Abafar,,,12 11 714 (polar diameter)
Aleen,399 days,31 hours,
Asmeru,At least sixteen months,,
Atollon,505 standard days,29.5 standard hours,
Auratera,280 days per year,18 hours per day,
Bardotta,608 days,25 hours,
Cato Neimoidia,278 local days,25 standard hours,
Devaron,298 days,19 hours,
Empress Teta,359 standard days,23.5 standard hours,
Kowak,311 standard days,23 standard hours,


There is single value without any unit. Unfortunatelly, quick check on wookieepedia don't settle this problem. For the purpose of this project we will assume most possible unit, which is kilometers.

Now to the lengthyear column. I've noticed houts, days, months and years. Let's check if I missed anything

In [26]:
swPlanets[~swPlanets['lengthyear'].str.contains('hour|day|month|year')]

,lengthyear,lengthday,diameter
Abafar,,,12 11 714 (polar diameter)
Eadu,,,"14,121 kilometers"
Endor (planet),,,"48,951 kilometers"
Kijimi,,,"9,874 km"
Lah'mu,,,"12,618 km"
Mygeeto,,,"10,088 kilometers"
Ord Mantell,,,"14,050 kilometers"
Saleucami,,,"14,920 kilometers"
Scarif,,,"9,112 km"
Stygeon Prime,,,"9,100 kilometers"


Nope, all checks out.

To go further with our processing we should examin all units to drew some conclusions:

**Hours:**

In [27]:
swPlanets[swPlanets['lengthyear'].str.contains('hour')]

,lengthyear,lengthday,diameter
Weik,30 hours per day,248 days per year,


I don't how about you but I am starting to think those values are swapped. If **Weik** has 30 hours in one day and 248 in one year, 30 hours could not possibly mean the length of it's year. It looks like someone made a mistake. Let's correct it.

In [28]:
temp = swPlanets.at['Weik','lengthyear'] 
swPlanets.at['Weik','lengthyear'] = swPlanets.at['Weik','lengthday']
swPlanets.at['Weik','lengthday'] = temp

**Days:**

In [29]:
swPlanets[swPlanets['lengthyear'].str.contains('day')]

,lengthyear,lengthday,diameter
Alderaan,364 standard days,18 standard hours,"12,500 kilometers"
Aleen,399 days,31 hours,
Atollon,505 standard days,29.5 standard hours,
Auratera,280 days per year,18 hours per day,
Bardotta,608 days,25 hours,
Batuu,365 days,24 hours,"12,750 kilometers"
Cato Neimoidia,278 local days,25 standard hours,
Corellia,329 standard days,25 standard hours,"11,000 kilometers"
Coruscant,365 standard days,24 standard hours,"12,240 kilometers"
Crait,525 standard days,27 standard hours,"7,400 kilometers"


**Months:**

In [30]:
swPlanets[swPlanets['lengthyear'].str.contains('month')]

,lengthyear,lengthday,diameter
Asmeru,At least sixteen months,,


We will need to corrrect this data, since it is not numerical. Let's just do this right now:

In [31]:
swPlanets.at['Asmeru','lengthyear'] = '16 months'

**Years:**

In [32]:
swPlanets[swPlanets['lengthyear'].str.contains('year')]

,lengthyear,lengthday,diameter
Auratera,280 days per year,18 hours per day,
Bespin,14 standard-years,12 standard-hours,"118,000 kilometers, (73,322 miles)"
Weik,248 days per year,30 hours per day,


There are several things we can conclude from this:
- **days** have priority over **years** (two representation of years, but after days if days are present)
- there is one representation of **months**
- we have data about **"local days"** and **"standard days"**. Local days means time the planet rotates around it's axis, while standard days (and "days") means standarized value in reference to our Earth day (24 hours)

There is no real value in analysing local days. There is a trick to make them worth though.
Let's see all values with "local" string first (or with left bracket "(" which could indicate second value)

In [33]:
swPlanets[swPlanets['lengthyear'].str.contains('local|\(')]

,lengthyear,lengthday,diameter
Cato Neimoidia,278 local days,25 standard hours,
Dantooine,378 local days,25 standard hours,"9,830 kilometers"
Exegol,210 local days,53 standard days,"13,649 km"
Hoth,"549 days, (526 standard days)",23 standard hours,"7,200 km, (4,474 miles)"
Ilum,"301 days, Variable (34 ABY)",66 hours,660 kilometers
Jakku,"352 standard days, 315 local days",26.8 standard hours,"6,400 kilometers"
Malastare,201 local days,26 standard hours,
Mustafar,412 local rotations (618 standard days),36 standard hours,"4,200 kilometers, (2,610 miles)"
Neimoidia,221 local days,29 standard hours,"10,830 kilometers"
Onderon,405 local days,28 standard hours,"15,190 kilometers"


And now I can finally explain why I decided to read lengthday column as well.

Knowing the value of planet's local day and planet's year in local days we can calculate the standard day.

The formula will look like this:

*lenghtday [h] * lengthyear [d] / 24 [h] = year [standard days]*

There is ONE planet that annoys me though. **Hoth**.

In [34]:
swPlanets.loc['Hoth']

lengthyear    549 days,  (526 standard days)
lengthday                  23 standard hours
diameter            7,200 km,  (4,474 miles)
Name: Hoth, dtype: object

To keep my data clean I will change it's first value to **local days** because information in bracket indicates that that's what it is.

In [35]:
swPlanets.at['Hoth','lengthyear'] = swPlanets.at['Hoth','lengthyear'].split()[0]+' local days'

So this is what we are going to do with lengthyear column:
- if string has both **'standard'** and **'local'** strings in it:
    - if **'local'** is <u>BEFORE</u> **'standard'** - unit is 'local days'
    - if **'local'** is <u>AFTER</u> **'standard'** - unit is 'standard days'
- if string has **'local'** and <u>not</u> **'standard'** in it - unit is 'local days'
- else unit is 'standard days'

After that we will **split** and **clear** value column, just as in exoplanets dataset, convert them into **float type** and use our **standard year formula** for values described as 'local days'.

In [36]:
def year_unit(year):
    if 'local' and 'standard' in str(year):
        if str(year).find('local') > str(year).find('standard'):
            return 'local days'
        return 'standard days'
    elif 'local' in str(year):
        return 'local days'
    elif 'days' in str(year):
        return 'standard days'
    elif 'months' in str(year):
        return 'months'
    elif 'years' in str(year):
        return 'years'
    return ''

In [37]:
swPlanets.insert(loc=1, column='lengthyearUnit', value=swPlanets['lengthyear'].apply(year_unit))
swPlanets.insert(loc=3, column='lengthdayUnit', value=swPlanets['lengthday'].apply(lambda i: i if i==''
                                                                                  else 'days' if 'days' in str(i)
                                                                                  else 'hours'))
swPlanets.insert(loc=5, column='diameterUnit', value=swPlanets['diameter'].apply(lambda i: i if i==''
                                                                                 else 'kilometers'))

We will prepare clearing function for diameter column now, since it is a little bit tricky (values separated by spaces or commas). After that we will apply it on diameter values and split lenghtyear and lengthday values to remain only numbers. After everything is prepared we will finally convert all of them to float type.

In [38]:
notNum3=[]
allData = list(swPlanets['diameter'])
for data in allData:
    for i in range(len(data)):
        if str(data)[i] not in '1234567890':
            notNum3.append(data[i])
notNum3 = pd.Series(notNum3)
notNumList3 = list(notNum3.unique())
#there is no need to remove deicmal point because there is none in the list

In [39]:
def clear_data(data):
    clearedData = []
    for i in range(len(data)):
        if data[i] not in notNumList3:
            clearedData.append(data[i])
    return ''.join(clearedData)

In [40]:
swPlanets['lengthyear'] = swPlanets['lengthyear'].apply(lambda year: year.split(' ')[0])
swPlanets['lengthday'] = swPlanets['lengthday'].apply(lambda day: day.split(' ')[0])
swPlanets['diameter'] = swPlanets['diameter'].apply(clear_data)
swPlanets['lengthyear'] = swPlanets['lengthyear'].apply(lambda i: i if i=='' else float(i))
swPlanets['lengthday'] = swPlanets['lengthday'].apply(lambda i: i if i=='' else float(i))
swPlanets['diameter'] = swPlanets['diameter'].apply(lambda i: i if i=='' else float(i))
swPlanets.head()

,lengthyear,lengthyearUnit,lengthday,lengthdayUnit,diameter,diameterUnit
Abafar,,,,,1211714.0,kilometers
Alderaan,364.0,standard days,18.0,hours,12500.0,kilometers
Aleen,399.0,standard days,31.0,hours,,
Asmeru,16.0,months,,,,
Atollon,505.0,standard days,29.5,hours,,


That's it.

The only thing left is to manipulate our value datas based on unit columns, to present them in the same, standarized form. We will handle this in Data preparation chapter

## Data preparation

The main thing we need to do now is to convert every value to the same unit.

I have decided to use **DAYS** for Orbital period / lengthyear and **KILOMETERS** for Mean radius / diameter.

We need to make several operations to do that:
- **swPlanets**: Imply our formula to calculate standard days to lengthyear values with 'local days' unit:
   - convert lengthday values to hours
   - define function that will imply our formula based on lengthdayUnit
   - apply definded function on lengthyear column
   - drop lenghtday and lengthdayUnit columns (we won't need them anymore)


- **swPlanets**: Standardize data:
   - convert lengthyear column to days based on lengthyearUnit
   - drop lengthyearUnit column
   - drop diameterUnit column (ALL diameter values are already in kilometers unit, so we don't need to make any conversions here)
   - rename columns to 'Orbital period [h]' and 'Diameter [km]'


- **exoPlanets**: Standardize data:
   - convert Orbital period column to days based on OrbitalPeriodUnit
   - convert Mean radius column to kilometers based on MeanRadiusUnit and values:
     - RJ (<a href="https://en.wikipedia.org/wiki/Jupiter_radius">Jupiter radius</a>) = 71492 km
     - RE (<a href="https://en.wikipedia.org/wiki/Earth_radius">Earth radius</a>) = 6378 km
   - convert Mean radius to Diameter (multiply value by 2)
   - drop OrbitalPeriodUnit column
   - drop MeanRadiusUnit column
   - rename columns to 'Orbital period [h]' and 'Diameter [km]'

In [41]:
#converting lengthday to hours
temp1 = swPlanets['lengthdayUnit'].apply(lambda i: 1 if i=='hours'
                                         else 24 if i=='days'
                                         else '')
swPlanets['lengthday'] = [swPlanets['lengthday'][i]*temp1[i] if temp1[i]!='' else '' for i in range(len(temp1))]
swPlanets.drop(['lengthdayUnit'], axis=1, inplace=True)

In [42]:
#defining function to convert local days to standard days
def convert_to_standard(yearValue, yearUnit, dayValue):
    standard = []
    for i in range(len(yearValue)):
        if 'local' in yearUnit[i]:
            standard.append(round(yearValue[i]*dayValue[i]/24,1))
        else:
            standard.append(yearValue[i])
    return standard

In [43]:
#implying function
swPlanets['lengthyear'] = convert_to_standard(swPlanets['lengthyear'],swPlanets['lengthyearUnit'],swPlanets['lengthday'])
swPlanets.drop(['lengthday'], axis=1, inplace=True)

In [44]:
#standardizing swPlanets data
temp = swPlanets['lengthyearUnit'].apply(lambda i: 1 if i=='days' 
                                         else 365/12 if i=='months' 
                                         else 365 if i=='years'
                                         else '')
swPlanets['lengthyear'] = [swPlanets['lengthyear'][i]*temp[i] if temp[i]!='' else '' for i in range(len(temp))]
swPlanets.drop(['lengthyearUnit','diameterUnit'], axis=1, inplace=True)
swPlanets.rename({'lengthyear':'Orbital period [d]','diameter':'Diameter [km]'}, axis=1, inplace=True)
swPlanets.head()

,Orbital period [d],Diameter [km]
Abafar,,1211714.0
Alderaan,,12500.0
Aleen,,
Asmeru,486.666667,
Atollon,,


In [45]:
#standardizing exoPlanets data
temp1 = exoPlanets['OrbitalPeriodUnit'].apply(lambda i: 1 if i=='days' else 365 if i=='years' else '')
temp2 = exoPlanets['RadiusUnit'].apply(lambda i: 71.492*2 if i=='RJ' else 6.378*2 if i=='RE' else '')
exoPlanets['Orbital period'] = [exoPlanets['Orbital period'][i]*temp1[i] if temp1[i]!='' else '' for i in range(len(temp1))]
exoPlanets['Mean radius'] = [exoPlanets['Mean radius'][i]*temp2[i] if temp2[i]!='' else '' for i in range(len(temp2))]
exoPlanets.drop(['OrbitalPeriodUnit','RadiusUnit'], axis=1, inplace=True)
exoPlanets.rename({'Orbital period':'Orbital period [d]','Mean radius':'Diameter [km]'}, axis=1, inplace=True)
exoPlanets.head()

,Orbital period [d],Diameter [km]
Kepler-42c,0.453287,9.31188
55 Cancri e,0.736547,23.9175
WASP-19b,0.78884,198.175824
WASP-43b,0.813478,148.70336
Kepler-10b,0.837495,18.75132


That's it! the only thing that remains is to export prepared dataframes to csv files:

In [46]:
swPlanets.to_csv('./Cleaned_StarWarsPlanets.csv', index = False)
exoPlanets.to_csv('./Cleaned_Exoplanets.csv', index = False)

## Summary

This was honestly the hardest and the most challanging part of this project. The data that we are working on (especially data that we scraped from the internet sites apparently) can have various forms and some can be really unorginized and hard to clean. It's not only about finding right formulas and functions to apply on those sets - it's about do and watch combination. Apply, watch how data react. Conlude, find most optimal solution and proceed to the next step. There isn't one universal way or one universal solution. You have to be adaptable when working on data. That's a valuable lesson.

Now with prepared cleaned data sets we can move to Part 3 of my project. There I will create an algorithm that will find exoplanet that is the most similar to chosen star wars planet, which will finally answer the question "Could Star Wars planets exist in our universe?".